In [99]:
from PIL import Image
import numpy as np
import pyresample as pr
from pycoast import ContourWriterAGG
import aggdraw
import pyproj

In [100]:
PNGFILE='lccn_TRGB_20210401_0230.png'
imgS = Image.open(PNGFILE) 

In [101]:
CntR,CntG,CntB=np.array(imgS.split())

In [124]:
area_id='LCC'
description='Plot LCC RGB file'
proj_id='lcc'
width=3000
height=3000
proj_args_lcc=" +proj=lcc +lat_0=10. +lat_1=30. +lat_2=60. +lon_0=128.5 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84"
prj=pyproj.Proj(proj_args_lcc)
([x1,x2],[y1,y2])=prj([102.,155.],[-1.,48.])
area_extent_lcc=(x1,y1,x2,y2)

Source_def = pr.AreaDefinition(area_id, description, proj_id, proj_args_lcc,width,height,area_extent_lcc)
Image_def = pr.load_area('../Table/Projection.proj','TW_800')  # 輸出圖檔的投影

In [125]:
def lcc2img(CntS,Source_def,Image_def):
    RoI = 5000 # 取樣之影響半徑
    Cnt = pr.kd_tree.resample_nearest(Source_def, CntS, Image_def, radius_of_influence=RoI)
    Img = Image.fromarray(Cnt,'L') #　轉成圖檔(灰階)
    return Img

In [126]:
imgR=lcc2img(CntR,Source_def,Image_def)
imgG=lcc2img(CntG,Source_def,Image_def)
imgB=lcc2img(CntB,Source_def,Image_def)



In [127]:
img=Image.merge('RGB',(imgR,imgG,imgB))
ShapePath = '../Table/gshhg-shp-2.3.7'
cw = ContourWriterAGG(ShapePath)
#cw.add_coastlines(img,Image_def,resolution='l',outline=(255,0,0),width=2)
cw.add_coastlines(img,Image_def,resolution='h',outline=(255,0,0),width=2)
ttfFile = '../Table/LiberationMono-Regular.ttf'
font = aggdraw.Font('white',ttfFile)
cw.add_grid(img,Image_def,(20,20),(10,10),font,outline=(0,0,255),width=3)
img.show()